# Landsat Compositing

This notebook explores creating composites (i.e., mosaics) from the Landsat Collection 2 data using:

- [Landsat Collection 2 STAC API](https://landsatlook.usgs.gov/stac-server), a catalog of Landsat data
- [pystac-client](https://pystac-client.readthedocs.io/) for searching and access data
- [OpenDataCube](https://www.opendatacube.org/) and [odc-stac](https://odc-stac.readthedocs.io/) for loading STAC assets and representing geospatial data as XArrays
- [XArray](http://xarray.pydata.org/en/stable/), [pandas](https://pandas.pydata.org/) [geopandas](https://geopandas.org/), [odc-tools](https://github.com/opendatacube/odc-tools), and [deafrica_tools](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/tree/main/Tools) for manipulating data
- [Dask](https://dask.org/) for performing parallel, distributed computing
- [Coiled.io](https://coiled.io/), a service for hosting Dask clusters
- [hvplot](https://hvplot.holoviz.org/) for visualization

Shown will be how to find data for an area of interest, explore the resulting metadata, perform calculations, and visualize the results.

In [ ]:
# initialize holoviews

import holoviews as hv
hv.extension('bokeh')

import geopandas as gpd
import hvplot.pandas
import pandas as pd

# Choose Area of Interest

In [ ]:
# AOIs available

from glob import glob
from pprint import pprint

pprint(glob("../aois/*"))

In [ ]:
# read the GeoJSON file and create a map

import json
from pathlib import Path

aoi_fname = "../aois/malawi.geojson"

aoi = json.loads(Path(aoi_fname).read_text())

# use folium to display vectors
# Several folium basemap tiles are available:
#   - OpenStreetMap
#   - Stamen Terrain
#   - Stamen Toner
#   - Stamen Watercolor
#   - CartoDB positron
#   - CartoDB dark_matter

import folium

map = folium.Map(tiles='OpenStreetMap')

# add vector to map, as transparent polygon
folium.GeoJson(aoi, style_function = lambda x: {'fillColor': '#00000000'}).add_to(map)

# fit the map to the bounds of the data
lons = [x[0] for x in aoi["geometry"]["coordinates"][0]]
lats = [x[1] for x in aoi["geometry"]["coordinates"][0]]
map.fit_bounds([(min(lats), min(lons)), (max(lats), max(lons))])

map

# Data Discovery

Use pystac-client to find data in the Landsat STAC API. First, fetch the collection of interest: Landsat Collection 2, Level 2 Surface Reflectance (landsat-c2l2-sr) and print the assets that are available. Then make a query with an AOI, date range, and search parameters.

The Landsat Items are all the asset URLS are updated to use the provided s3 URLs which can be used for direct access rather than the default https URLs. This is because the alternate extension is not yet supported in PySTAC, when it is there will be an easier way to specify which alternate URL, if any, to use for the assets.

In [ ]:
from pystac_client import Client

api = Client.open("https://landsatlook.usgs.gov/stac-server")

collection = api.get_collection("landsat-c2l2-sr")
collection

In [ ]:
%%time

# search the API

query = api.search(
    collections=[collection.id],
    intersects=aoi['geometry'],
    datetime="2021-06-01/2021-08-31",
    limit=100,
    query = [
        "platform=LANDSAT_8",
        "eo:cloud_cover<80"
    ]
)
item_collection = query.item_collection()

print(f"Found: {len(item_collection):d} STAC Items")

In [ ]:
%%time

# display the map with footprints

# view footprints
style = {
    'fillColor': '#00000000', # transparent
    'color': '#fc0f03',       # red
    'weight': 1
}

for item in item_collection:
    folium.GeoJson(item.to_dict(), style_function=lambda x: style).add_to(map)

map

In [ ]:
# update Item assets to use the alternate s3 URL

def update_landsat_items(items_dict):
    # update URLs to use s3
    for item in items_dict:
        #print(item)
        for a in item['assets']:
            if 'alternate' in item['assets'][a] and 's3' in item['assets'][a]['alternate']:
                item['assets'][a]['href'] = item['assets'][a]['alternate']['s3']['href']
            item['assets'][a]['href'] = item['assets'][a]['href'].replace('usgs-landsat-ard', 'usgs-landsat')
    return items_dict

items_dict = update_landsat_items(item_collection.to_dict()['features'])

# OpenDataCube

Now we'll turn the set of scenes into a virtual datacube. None of the data will actually be read yet. A PySTAC ItemCollection is created from the found STAC Items, with parameters such as bands of interest and chunk size. The configuration string (`cfg`) is for providing info on the cloud mask values, which while in the the STAC Items, is not currently supported by ogc-stac.

In [ ]:
# read in additional Landsat config info

import yaml

def read_yaml(filename):
    with open(filename, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

cfg = read_yaml('landsat.yml')

In [ ]:
%%time

from odc.stac import stac_load

# default to CRS and resolution from first Item
from pystac.extensions.projection import ProjectionExtension
from pyproj import CRS

proj = ProjectionExtension.ext(item_collection[0])
output_crs = CRS.from_epsg(proj.epsg)
resolution = min(proj.transform[4], proj.transform[0])

dc = stac_load(item_collection,
               geopolygon=aoi['geometry'],
               chunks={"x": 2048, "y": 2048},
               output_crs=output_crs,
               resolution=resolution,
               groupby='solar_day',
               stac_cfg=cfg)
dc

In [ ]:
from odc.algo import erase_bad
from datacube.utils import masking

dc['qa_pixel'].attrs['flags_definition'] = cfg[collection.id]['assets']['qa_pixel']['flags_definition']

# remove negative pixels - a pixel is invalid if any of the band is smaller than masking_scale
#valid = (dc[self.bands] > (-1.0 * self.offset/self.scale)).to_array(dim='band').all(dim='band')

mask_band = dc['qa_pixel']
dc = dc.drop_vars(['qa_pixel'])

flags_def = masking.get_flags_def(mask_band)

# set cloud_mask - True=cloud, False=non-cloud
mask, _ = masking.create_mask_value(flags_def, cloud="high_confidence", cirrus="high_confidence",)
cloud_mask = (mask_band & mask) != 0

# set no_data bitmask - True=data, False=no-data
nodata_mask, _ = masking.create_mask_value(flags_def, nodata=False)
keeps = (mask_band & nodata_mask) == 0

dc = erase_bad(dc, where=cloud_mask)
dc['cloud_mask'] = cloud_mask

dc

# Start Dask Client

Start either a local Dask, or use [coiled.io](coiled.io)

In [ ]:
%%time

# start Dask cluster using coiled

import coiled
from dask.distributed import Client

# start dask cluster on coiled.io
cluster = coiled.Cluster(
    n_workers=10,
    software="cng-workshop",
    backend_options={"region": "us-west-2"},
    environ={"GDAL_DISABLE_READDIR_ON_OPEN": "YES", "AWS_REQUEST_PAYER": "requester"}
)
client = Client(cluster)

print('Dashboard:', client.dashboard_link)
client

# Visualize

Look at the individual time scenes and create an animated GIF

In [ ]:
%%time

from dask.distributed import wait

# persist the read data on Dask cluster
dc = client.persist(dc)
_ = wait(dc)

In [ ]:
#%%time

# fetch all the values from dask
#dc_local = dc.compute()

In [ ]:
%%time

from deafrica_tools.plotting import rgb

rgb(dc, bands=['red', 'green', 'blue'], col='time') #, percentile_stretch=[0.01, 0.99])

In [ ]:
%%time

from deafrica_tools.plotting import xr_animation
import matplotlib.pyplot as plt
from IPython.display import Image

# Produce time series animation of red, green and blue bands
xr_animation(ds=dc,
             bands=['red', 'green', 'blue'],
             output_path='landsat-timeseries.gif',
             percentile_stretch=(0.1, 0.99),
             interval=600,
             width_pixels=800)

# Plot animated gif
plt.close()
Image(filename='landsat-timeseries.gif')

In [ ]:
%%time

# persist the read data on Dask cluster
dc = client.persist(dc)
_ = wait(dc)

In [ ]:
%%time

from deafrica_tools.plotting import rgb

rgb(dc, bands=['red', 'green', 'blue'], col='time') #, percentile_stretch=[0.01, 0.99])

# Basic Composite

Generate basic composites using simple statistics: mean, median, min, max

In [ ]:
from odc.algo import to_float

dc_float = to_float(dc, dtype='float32')

rgb(dc_float.mean('time'), bands=['red', 'green', 'blue'], size=20, percentile_stretch=(0.02, 0.9))

In [ ]:
rgb(dc_float.median('time'), bands=['red', 'green', 'blue'], size=20, percentile_stretch=(0.02, 0.92))

# Percentile Composite

The datacube currently contains complete Items, we want to clip these to our geometry of interest. We will then also create an RGB datacube representation, and generate an NDVI datacube.

In [ ]:
from odc.algo import xr_quantile

stats = xr_quantile(dc, [0.2, 0.8], 0.0)

stats = stats.compute()

stats

In [ ]:
%%time

rgb(stats.sel(quantile=0.2), bands=['red', 'green', 'blue'], size=12, percentile_stretch=[0.02, 0.98])

In [ ]:
import hvplot.xarray

from odc.algo import to_rgba

epsg = int(dc.attrs['crs'].split(':')[-1])

vis = to_rgba(stats.sel(quantile=0.8), clamp=(1000, 30000), bands=['red', 'green', 'blue'])
vis.hvplot.rgb(x='x', y='y', bands='band', crs=epsg, tiles='OSM', frame_width=800)

# Shutdown cluster

In [ ]:
client.close()
cluster.shutdown()
cluster.close()